In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('data/atx_crash_data_2018-2026.csv')

C:\Users\eawil\AppData\Local\Temp\ipykernel_22500\848549886.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/atx_crash_data_2018-2026.csv')


In [3]:
df.head()

,ID,Crash ID,crash_fatal_fl,case_id,rpt_block_num,rpt_street_name,rpt_street_sfx,crash_speed_limit,road_constr_zone_fl,latitude,longitude,crash_sev_id,units_involved,point,onsys_fl,private_dr_fl,Crash timestamp (US/Central),Estimated Total Comprehensive Cost,Location group,Address
0,246606,16209332.0,False,180010160,12300,HARRIS BRANCH,PKWY,45.0,False,30.368405,-97.613322,5,Passenger car,POINT (-97.61332249 30.36840475),True,False,1/1/2018 0:36,60000,1.0,N HARRIS BRANCH PKWY & E PARMER LN
1,245425,16182657.0,False,180010354,14300,IH 35,NaN,50.0,False,30.433927,-97.670054,3,Passenger car,POINT (-97.67005447 30.43392675),False,False,1/1/2018 1:36,440000,1.0,14300 N IH 35
2,245426,16182659.0,False,180010462,8300,BEN WHITE,BLVD,55.0,False,30.222877,-97.679100,0,Passenger car,POINT (-97.67910026 30.2228774),True,False,1/1/2018 2:26,20000,2.0,8300 E STATE HIGHWAY 71
3,245514,16185982.0,False,180010524,8400,SH 71 WB,NaN,45.0,False,30.247853,-97.889753,5,Passenger car,POINT (-97.889753 30.247853),True,False,1/1/2018 2:43,20000,1.0,8400 E STATE HIGHWAY 71
4,245513,16185981.0,False,180010550,5800,LARK CREEK DR,DR,30.0,False,30.192214,-97.731125,5,Passenger car,POINT (-97.73112546 30.1922137),False,False,1/1/2018 2:51,20000,1.0,5800 LARK CREEK DR


In [5]:
def extract_units(s):
    a_list = []
    for units in s.split('&'):
        for unit in units.split('â€“'):
            a_list.append(unit.strip())

    return a_list

unit_cats = list(set(i for s in df['units_involved'].str.lower().map(extract_units).to_list() for i in s))
unit_cats.remove('other/unknown') # combine with "other"
unit_cats

['bicycle',
 'train',
 'motor vehicle',
 'large passenger vehicle',
 'pedestrian',
 'micromobility device',
 'e-scooter',
 'other',
 'motorcycle',
 'passenger car']

In [6]:
df['day_of_week'] = pd.to_datetime(df['Crash timestamp (US/Central)']).dt.weekday
df['week_of_year'] = pd.to_datetime(df['Crash timestamp (US/Central)']).dt.isocalendar().week
df['hour_of_day'] = pd.to_datetime(df['Crash timestamp (US/Central)']).dt.hour

df['units_involved'] = df['units_involved'].str.lower()
for cat in unit_cats:
    df[f'{cat}_involved'] = df['units_involved'].str.contains(cat)

In [11]:
df.drop(columns=['Crash timestamp (US/Central)','units_involved'],inplace=True)
df.dropna(inplace=True)

In [12]:
df.head()

,ID,Crash ID,crash_fatal_fl,case_id,rpt_block_num,rpt_street_name,rpt_street_sfx,crash_speed_limit,road_constr_zone_fl,latitude,longitude,crash_sev_id,point,onsys_fl,private_dr_fl,Estimated Total Comprehensive Cost,Location group,Address,day_of_week,week_of_year,hour_of_day,bicycle_involved,train_involved,motor vehicle_involved,large passenger vehicle_involved,pedestrian_involved,micromobility device_involved,e-scooter_involved,other_involved,motorcycle_involved,passenger car_involved
0,246606,16209332.0,False,180010160,12300,HARRIS BRANCH,PKWY,45.0,False,30.368405,-97.613322,5,POINT (-97.61332249 30.36840475),True,False,60000,1.0,N HARRIS BRANCH PKWY & E PARMER LN,0,1,0,False,False,False,False,False,False,False,False,False,True
2,245426,16182659.0,False,180010462,8300,BEN WHITE,BLVD,55.0,False,30.222877,-97.679100,0,POINT (-97.67910026 30.2228774),True,False,20000,2.0,8300 E STATE HIGHWAY 71,0,1,2,False,False,False,False,False,False,False,False,False,True
4,245513,16185981.0,False,180010550,5800,LARK CREEK DR,DR,30.0,False,30.192214,-97.731125,5,POINT (-97.73112546 30.1922137),False,False,20000,1.0,5800 LARK CREEK DR,0,1,2,False,False,False,False,False,False,False,False,False,True
5,245368,16180819.0,False,180010555,1700,E PARMER LN,LN,60.0,False,30.386948,-97.648715,0,POINT (-97.648714578022 30.3869478988931),True,False,60000,1.0,E PARMER LN & DESSAU RD,0,1,2,False,False,False,False,False,False,False,False,False,True
8,278237,17979600.0,False,180010873,700,PARMER,LN,60.0,False,30.405337,-97.664316,5,POINT (-97.6643163 30.40533691),True,False,40000,1.0,700 E FM 734,0,1,7,False,False,False,False,False,False,False,False,False,True


In [8]:
df.to_csv('atx_crash_data_2018-2026_cleansed.csv', index=False)

In [9]:
df_filetest= pd.read_csv('data/atx_crash_data_2018-2026_cleansed.csv') 

C:\Users\eawil\AppData\Local\Temp\ipykernel_22500\3199035996.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_filetest= pd.read_csv('data/atx_crash_data_2018-2026_cleansed.csv')


In [10]:
df_filetest.head()

,ID,Crash ID,crash_fatal_fl,case_id,rpt_block_num,rpt_street_name,rpt_street_sfx,crash_speed_limit,road_constr_zone_fl,latitude,longitude,crash_sev_id,point,onsys_fl,private_dr_fl,Crash timestamp (US/Central),Estimated Total Comprehensive Cost,Location group,Address,day_of_week,week_of_year,hour_of_day,micromobility device_involved,e-scooter_involved,other_involved,passenger car_involved,bicycle_involved,large passenger vehicle_involved,train_involved,motor vehicle_involved,motorcycle_involved,pedestrian_involved
0,246606,16209332.0,False,180010160,12300,HARRIS BRANCH,PKWY,45.0,False,30.368405,-97.613322,5,POINT (-97.61332249 30.36840475),True,False,1/1/2018 0:36,60000,1.0,N HARRIS BRANCH PKWY & E PARMER LN,0,1,0,False,False,False,True,False,False,False,False,False,False
1,245426,16182659.0,False,180010462,8300,BEN WHITE,BLVD,55.0,False,30.222877,-97.679100,0,POINT (-97.67910026 30.2228774),True,False,1/1/2018 2:26,20000,2.0,8300 E STATE HIGHWAY 71,0,1,2,False,False,False,True,False,False,False,False,False,False
2,245513,16185981.0,False,180010550,5800,LARK CREEK DR,DR,30.0,False,30.192214,-97.731125,5,POINT (-97.73112546 30.1922137),False,False,1/1/2018 2:51,20000,1.0,5800 LARK CREEK DR,0,1,2,False,False,False,True,False,False,False,False,False,False
3,245368,16180819.0,False,180010555,1700,E PARMER LN,LN,60.0,False,30.386948,-97.648715,0,POINT (-97.648714578022 30.3869478988931),True,False,1/1/2018 2:52,60000,1.0,E PARMER LN & DESSAU RD,0,1,2,False,False,False,True,False,False,False,False,False,False
4,278237,17979600.0,False,180010873,700,PARMER,LN,60.0,False,30.405337,-97.664316,5,POINT (-97.6643163 30.40533691),True,False,1/1/2018 7:28,40000,1.0,700 E FM 734,0,1,7,False,False,False,True,False,False,False,False,False,False
